Lakeflow Job 動的値参照
https://docs.databricks.com/aws/en/jobs/dynamic-value-references

In [13]:
%pip install yfinance



[notice] A new release of pip is available: 24.0 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import yfinance as yf


In [ ]:
# import sys
# sys.path.append('/Workspace/Users/databricks-admin@temp389.onmicrosoft.com/.bundle/stock_market_anaysis/dev/files')

# from src.utils.utils import add_one_day


In [ ]:
# %pip install -e /Workspace/Users/databricks-admin@temp389.onmicrosoft.com/.bundle/stock_market_anaysis/dev/files

# from src.utils.utils import add_one_day


In [ ]:
stock_price_date = dbutils.widgets.get("stock_price_date")
directory_path = dbutils.widgets.get("directory_path")


In [23]:
stock_master_df = spark.read.table("`yfinance-dev`.silver.stock_master")


In [ ]:
code_list = [f"{code}.T" for code in stock_master_df.toPandas()["code"].tolist()]


In [45]:
tickers = yf.Tickers(code_list)


In [ ]:
stock_price_pdf = yf.download(code_list, start=stock_price_date, end=add_one_day(stock_price_date), group_by="ticker")


[*********************100%***********************]  1 of 1 completed


In [49]:
data_list = []
for code in code_list:
    temp_df = stock_price_pdf[code].copy()
    temp_df["Ticker"] = code
    temp_df = temp_df.reset_index()
    data_list.append(temp_df)

converted_stock_price_pdf = pd.concat(data_list)


In [ ]:
df = spark.createDataFrame(converted_stock_price_pdf)


In [ ]:
df.write.mode("overwrite").option("header", "true").parquet(f"{directory_path}/stock_price_{stock_price_date}")
